In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from collections import OrderedDict

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Times New Roman']
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
H_G1 = 500
stoch_load_bus_IDs = [3]
rec_bus_IDs = []
D = 2
DZA = 0

experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=['ReLU_none'], \
                                  verbose=True)
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=True)
data_dirs = ['..' + os.path.sep +
             os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d]) +
             f'/H_G1_{H_G1}/stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
             for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names']]
var_names_fmt = list(OrderedDict({k + '{}': [] for k in tmp}).keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

In [ ]:
default_H = OrderedDict([
    ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
    ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42)
])

generators_areas_map = {
    'default': [
        ['Pg31', 'Pg32', 'Pg39'],
        ['Pg33', 'Pg34', 'Pg35', 'Pg36'],
        ['Pg37', 'Pg38'],
        ['Pg30']
    ],
    'compensator': [
        ['Pg31', 'Pg32', 'Pg39', 'Pg99'],
        ['Pg33', 'Pg34', 'Pg35', 'Pg36'],
        ['Pg37', 'Pg38'],
        ['Pg30']
    ]
}

P_nom = {gen_ID: 100e6 for gen_ID in default_H}

window_dur = 60
window_step = 1

var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

if area_measure == 'inertia':
    measure_units = 's'
elif area_measure == 'energy':
    measure_units = r'GW$\cdot$s'
elif area_measure == 'momentum':
    measure_units = r'GW$\cdot$s$^2$'
    
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))

abbrv = {'inertia': 'H', 'energy': 'E', 'momentum': 'M'}

## Constant measure

In [ ]:
dead_zone_amplitude = 0.0
if dead_zone_amplitude != DZA:
    data_dir = data_dir.replace(f'{DZA:.1f}',f'{dead_zone_amplitude:.1f}')
    print(f'Loading data from {data_dir}.')

H_values = [
    default_H,
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 34), ('Pg32', 40), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 46)

    ])
]
N_H = len(H_values)

measure_exact = []

data_normalized = []
measure = []

var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

for H in H_values:
    data_file = data_dir + '/ieee39_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    measure_exact.append(v[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(8, n_vars * 2.5))
gs = fig.add_gridspec(n_vars+1, 4)
ax = []
for i in range(n_vars):
    ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])]),
ax.append([fig.add_subplot(gs[-1, :2]), fig.add_subplot(gs[-1, 2:])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0]]

bus_ID = rec_bus_IDs[0]

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        key = var_name.format(bus_ID)
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=col[i], lw=1, \
                      label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][i].plot(time / 60, measure[i], 'k', lw=1)
    ax[-1][i].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), 'k--', lw=1)
    ax[-1][i].set_ylim(ylim)
    ax[-1][i].set_xlabel('Time [min]')

for a in ax:
    for side in 'right','top':
        for i in range(2):
            a[i].spines[side].set_visible(False)

ax[0][0].legend(loc='best')
ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][1])

ax[-1][0].set_xlim([0,30])
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_const_H_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)

## Step of measure with transient

In [ ]:
H_values = [
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 20), ('Pg32', 28), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 36)

    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 42), ('Pg32', 46), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 50)

    ])
]

measure_exact = []
for H_val in H_values:
    if area_measure == 'inertia':
        pass
    elif area_measure == 'energy':
        measure_exact.append(np.sum([H_val[gen_id] * P_nom[gen_id] for gen_id \
                                     in generators_areas_map['default'][area_ID - 1]]) * 1e-9)
    elif area_measure == 'momentum':
        measure_exact.append(2 * np.sum([H_val[gen_id] * P_nom[gen_id] for gen_id \
                                         in generators_areas_map['default'][area_ID - 1]]) * 1e-9 / 60)

data_file = data_dir + '/ieee39'
for gen_id in H_values[0]:
    data_file += '_' + '-'.join(map(lambda h: f'{h:.3f}', np.unique([H[gen_id] for H in H_values])))
data_file += '.h5'

t, _, data_normalized, data_sliding, _ = load_data_slide([data_file],
                                                         var_names,
                                                         data_mean,
                                                         data_std,
                                                         window_dur,
                                                         window_step,
                                                         add_omega_ref = False,
                                                         verbose = True)

dt = np.diff(t[:2])[0]
time, measure, _ = predict(model, data_sliding, window_step)

In [ ]:
fig = plt.figure(figsize = (6,9))
gs = fig.add_gridspec(2 * n_vars + 3 + 1, 1)
ax = []
for i in range(n_vars):
    ax.append(fig.add_subplot(gs[i*2:(i+1)*2]))
ax.append(fig.add_subplot(gs[-4:-1,0]))
ax.append(fig.add_subplot(gs[-1,0]))

ds = 5
for i,var_name in enumerate(var_names_fmt):
    for j,bus_id in enumerate(rec_bus_IDs[::-1]):
        ax[i].plot(t[::ds] / 60, data_normalized[var_name.format(bus_id)][::ds],
                   color=col[j], lw=1, label=f'bus{bus_id}')
    ax[i].set_ylim([-5,5])
    ax[i].set_ylabel(var_name.split('_')[0])

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 3, np.max(measure_exact) + dm / 3]

ax[-2].plot(time[::ds] / 60, measure[::ds,0], color=col[0])
ax[-2].set_ylim(ylim)
ax[-2].plot([0, 30], measure_exact[0] + np.zeros(2), '--', color=col[0], lw=1)
ax[-2].plot([30, 60], measure_exact[1] + np.zeros(2), '--', color=col[0], lw=1)

ax[-1].plot([0, 30], measure_exact[0] + np.zeros(2), '-', color=col[0], lw=2)
ax[-1].plot([30, 60], measure_exact[1] + np.zeros(2), '-', color=col[0], lw=2)
ax[-1].plot(30 + np.zeros(2), ylim, '--', color=col[0], lw=1)
ax[-1].set_ylim(ylim)
# ax[-1].set_yticks(np.arange(8,15,2))
ax[-1].set_xlabel('Time [min]')

ax[0].legend(loc='lower right')

for a in ax:
    a.set_xlim([-1,61])
    for side in 'right', 'top':
        a.spines[side].set_visible(False)
    if a != ax[-1]:
        a.set_xticklabels([])

ax[-2].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-1].set_ylabel(f'{abbrv[area_measure]} [{measure_units}]')

fig.tight_layout()
output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_step_H.pdf'
fig.savefig(output_filename)

## With a 4th generator (a compensator) added in area 1

In [ ]:
P_nom['Pg99'] = 100e6

H_values = [
    default_H,
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42),
        ('Pg99', 10)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42),
        ('Pg99', 20)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42),
        ('Pg99', 40)
    ])
]
N_H = len(H_values)

data_normalized = []
measure = []
measure_exact = []
for H in H_values:
    if 'Pg99' in H:
        data_file = data_dir + '/ieee39_compensator_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['compensator'], P_nom, area_measure)
    else:
        data_file = data_dir + '/ieee39_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)

    measure_exact.append(v[area_ID - 1])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, pred, _ = predict(model, data_sliding, window_step)
    measure.append(pred)

In [ ]:
measure_exact

In [ ]:
[np.nanmean(m) for m in measure]

In [ ]:
n_vars = len(var_names_fmt)
fig = plt.figure(figsize=(9, n_vars * 3))
gs = fig.add_gridspec(n_vars+1, 4)
ax = []
for i in range(n_vars):
    ax.append([fig.add_subplot(gs[i, :3]), fig.add_subplot(gs[i, 3])])
ax.append([fig.add_subplot(gs[-1, :])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0], [0,0,.7]]
cmap = plt.get_cmap('jet', N_H+1)
bus_ID = rec_bus_IDs[0]

idx = t < 60

dm = np.max(measure_exact) - np.min(measure_exact)
ylim = [np.min(measure_exact) - dm / 2, np.max(measure_exact) + dm / 2]

for i in range(N_H):
    for j,var_name in enumerate(var_names_fmt):
        key = var_name.format(bus_ID)
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=cmap(i), lw=1, \
                      label=f'{abbrv[area_measure]} = {measure_exact[i]:.2f} {measure_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=cmap(i), lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
        ax[j][0].set_ylabel(key)
    ax[-1][0].plot(time / 60, measure[i], color=col[i], lw=1)
    ax[-1][0].plot(time[[0,-1]] / 60, measure_exact[i] + np.zeros(2), '--', color=col[i], lw=1)
    ax[-1][0].set_ylim(ylim)
    ax[-1][0].set_xlabel('Time [min]')

for aa in ax:
    for side in 'right','top':
        for a in aa:
            a.spines[side].set_visible(False)
for i in range(n_vars):
    for j in range(2):
        ax[i][j].grid(axis='y', lw=0.5, linestyle=':')

ax[0][0].legend(loc='best')
ax[-1][0].set_xlim([0,60])
ax[-1][0].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
output_filename = f'IEEE39_area{area_ID}_H_G1={H_G1}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_compensator_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)